In [1]:
import numpy as np
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [3]:
class Neuron_sigmoid:
  def __init__(self, nin):
    self.w = np.random.rand(nin , 1)
    self.w = self.w/np.sqrt(nin)
    self.b = np.random.rand()
    self.d_W = np.zeros(self.w.shape)
    self.d_b = 0

  def __call__(self, x):
    self.X = x
    self.d_X = np.zeros(self.X.shape)
    act = np.dot(self.X,self.w)
    act = act + self.b
    self.out = sigmoid(act)
    return self.out

  def parameters(self):
    return self.w , self.b

  def _backward(self , d_loss_w_out, alpha, lamda):
    for i in range(0,self.X.shape[0]):
      for j in range(0,self.w.shape[0]):
        self.d_W[j][0] += self.out[i][0]*(1-self.out[i][0])*self.X[i][j]*d_loss_w_out[i][0]
        self.d_X[i][j] += self.out[i][0]*(1-self.out[i][0])*self.w[j][0]*d_loss_w_out[i][0]
      self.d_b += self.out[i][0]*(1-self.out[i][0])*d_loss_w_out[i][0]

    self.w = self.w - alpha*self.d_W - ((alpha*lamda)/self.X.shape[0])*self.w
    self.b = self.b - alpha*self.d_b

    self.d_W = np.zeros(self.w.shape)
    ret = self.d_X.copy()
    self.d_X = np.zeros(self.X.shape)
    self.d_b = 0

    return ret

In [4]:
class Layer_sigmoid:
  def __init__(self, nin, nout):
    self.neurons = [Neuron_sigmoid(nin) for _ in range(nout)]

  def __call__(self, x):
    self.outs = self.neurons[0](x)
    for i in range(1,len(self.neurons)):
      self.outs = np.concatenate((self.outs, self.neurons[i](x)), axis=1)
    return self.outs

  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

  def _backward(self, d_out, alpha, lamda):
    grad = 0
    for i in range(0,len(self.neurons)):
      grad_col = np.transpose([d_out[:,i]])
      output_neuron = np.transpose([self.outs[:,i]])
      grad += self.neurons[i]._backward(grad_col,alpha,lamda)

    return grad

In [5]:
class Neuron_softmax:
  def __init__(self, nin):
    self.w = np.random.rand(nin , 1)
    self.w = self.w/np.sqrt(nin)
    self.b = np.random.rand()
    self.d_W = np.zeros(self.w.shape)
    self.d_b = 0

  def __call__(self, x):
    self.X = x
    self.d_X = np.zeros(self.X.shape)
    act = np.dot(self.X,self.w)
    act = act + self.b
    self.out = np.exp(act)
    return self.out

  def parameters(self):
    return self.w , self.b

  def _backward(self , d_loss_w_out, alpha, lamda):
    for i in range(0,self.X.shape[0]):
      for j in range(0,self.w.shape[0]):
        self.d_W[j][0] += self.out[i][0]*self.X[i][j]*d_loss_w_out[i][0]
        self.d_X[i][j] += self.out[i][0]*self.w[j][0]*d_loss_w_out[i][0]
      self.d_b += self.out[i][0]*d_loss_w_out[i][0]

    self.w = self.w - alpha*self.d_W - ((alpha*lamda)/self.X.shape[0])*self.w
    self.b = self.b - alpha*self.d_b

    self.d_W = np.zeros(self.w.shape)
    ret = self.d_X.copy()
    self.d_X = np.zeros(self.X.shape)
    self.d_b = 0

    return ret

In [6]:
class Layer_softmax:
  def __init__(self, nin, nout):
    self.neurons = [Neuron_softmax(nin) for _ in range(nout)]

  def __call__(self, x):
    self.outs = self.neurons[0](x)
    for i in range(1,len(self.neurons)):
      self.outs = np.concatenate((self.outs, self.neurons[i](x)), axis=1)

    row_sums =[np.sum(self.outs, axis=1)]
    row_sums = np.transpose(row_sums)

    self.outs = self.outs/row_sums
    return self.outs

  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

  def _loss(self, Y):
    outputs = -np.log(self.outs)
    outputs = outputs*Y
    loss = np.sum(outputs)
    loss = loss/Y.shape[0]
    return loss

  def _backward(self,Y, alpha, lamda):
    d_out = (-((self.outs - np.square(self.outs))*Y)/Y.shape[0])

    grad = 0
    for i in range(0,len(self.neurons)):
      grad_col = np.transpose([d_out[:,i]])
      grad += self.neurons[i]._backward(grad_col,alpha,lamda)

    return grad

In [8]:
df = pd.read_csv("/content/train.csv")

In [9]:
df.head(10)

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1
5,5,1009,1033,899231,899307,409,22,26,47513,86,...,0.5909,-0.1890,0.8749,0,0,0,0,0,0,1
6,6,596,607,739072,7390760,204,21,21,22478,89,...,0.3158,-0.1497,0.5212,0,0,0,0,0,1,0
7,7,1673,1687,294065,294091,571,38,57,53142,77,...,0.0357,-0.2661,0.9408,1,0,0,0,0,0,0
8,8,507,521,203252,203261,101,25,12,12530,124,...,-0.6667,0.0305,0.3601,0,0,0,0,0,0,0
9,9,893,907,1341292,1341296,54,6,15,7052,87,...,0.4667,-0.1228,0.1400,0,0,0,0,0,1,0


In [13]:
X = df[['X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum',
        'Pixels_Areas', 'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity',
        'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer',
        'TypeOfSteel_A300', 'TypeOfSteel_A400', 'Steel_Plate_Thickness',
        'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index',
        'Edges_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas',
        'Log_X_Index', 'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index',
        'SigmoidOfAreas']]
X = np.array(X.values)
m = X.shape[0]
input_features = X.shape[1]
mean = np.sum(X, axis=0)
mean = mean/m
X = X-mean
var = np.var(X, axis=0)
X = X/var
Y = df[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]
Y = np.array(Y.values)
no_output_units = Y.shape[1]
lay1 = Layer_sigmoid(input_features,50)
lay2 = Layer_sigmoid(50, 30)
lay3 = Layer_softmax(30, no_output_units)

In [14]:
for _ in range(0,3):
  rows_per_matrix = 1000
  num_matrices = X.shape[0] // rows_per_matrix
  total_loss = 0


  for i in range(num_matrices):
    start_idx = i * rows_per_matrix
    end_idx = start_idx + rows_per_matrix
    temp_X = X[start_idx:end_idx, :]
    temp_Y = Y[start_idx:end_idx, :]
    output1 = lay1(temp_X)
    output2 = lay2(output1)
    output3 = lay3(output2)
    loss = lay3._loss(temp_Y)
    total_loss+=loss
    print(loss)
    alpha = 0.00001
    lamda = 0
    grad1 = lay3._backward(temp_Y,alpha, lamda)
    grad2 = lay2._backward(grad1, alpha, lamda)
    lay1._backward(grad2, alpha, lamda)

  print("----------exit------------",total_loss)

1.8457348212163278
1.847924658814195
1.845757662868872
1.8647658561762446
1.8411904875143725
1.8356220408906203
1.8663872364072682
1.8538939707085307
1.8149811284287498
1.8445517504848494
1.8657457504815371
1.842122579678622
1.8321697565493273
1.8958509645946573
1.85588455742891
1.8357262364440357
1.8369007442617598
1.8968270242241208
1.819871239230757
----------exit------------ 35.14190846640376
1.8448910552589877
1.847099833180654
1.8449149837148984
1.8637380253224896
1.8400654789106774
1.8346565675152529
1.8657102351426893
1.8529226055499923
1.813982582965411
1.8437392312254857
1.8648953925735845
1.841225485572866
1.8313097549152737
1.894944612565039
1.8550228162099869
1.8347486018056818
1.8359144907410354
1.8961922450586326
1.8189553069245048
----------exit------------ 35.124929305153145
1.8440490468926083
1.846277006118055
1.844074233023393
1.862709854251217
1.838939121120925
1.8336913127068852
1.8650370650707355
1.8519513198750512
1.812983774019461
1.8429290443873902
1.8640471914